In [2]:
import pandas as pd 
import numpy as np

In [16]:
df1 = pd.DataFrame({"a":[1,2,3], "b":[2,4,5]})
df2 = pd.DataFrame({"a":[5,2,5], "b":[4,3,2]})
print(f"df1\n{df1}")
print(f"df2\n{df2}")

df1
   a  b
0  1  2
1  2  4
2  3  5
df2
   a  b
0  5  4
1  2  3
2  5  2


In [49]:

keys= zip(range(2), [0.5, 0.2])
df_cat = pd.concat([df1, df2], keys=keys, names=["set", "label", "event"])
df_cat

a  b
set label event      
0   0.5   0      1  2
          1      2  4
          2      3  5
1   0.2   0      5  4
          1      2  3
          2      5  2

In [50]:
df_cat.index.get_level_values("set")[::len(df_cat.loc[0])].to_numpy()

array([0, 1])

In [51]:
len(df_cat.loc[0])

3

In [55]:
np.concatenate(
    [np.expand_dims(s, axis=0) for _, s in df_cat.groupby(level="set")],
    axis=0
)


array([[[1, 2],
        [2, 4],
        [3, 5]],

       [[5, 4],
        [2, 3],
        [5, 2]]])

In [78]:
df_cat.loc(level="event")

TypeError: _LocationIndexer.__call__() got an unexpected keyword argument 'level'

In [64]:
list_of_sets = [df_cat.loc[i] for i in df_cat.index.get_level_values("set").unique()]
list_of_sets

[             a  b
 label event      
 0.5   0      1  2
       1      2  4
       2      3  5,
              a  b
 label event      
 0.2   0      5  4
       1      2  3
       2      5  2]

In [66]:
expanded_list_of_sets = [np.expand_dims(s, axis=0) for s in list_of_sets]
expanded_list_of_sets

[array([[[1, 2],
         [2, 4],
         [3, 5]]]),
 array([[[5, 4],
         [2, 3],
         [5, 2]]])]

In [68]:
np_array = np.concatenate(expanded_list_of_sets, axis=0)
np_array

array([[[1, 2],
        [2, 4],
        [3, 5]],

       [[5, 4],
        [2, 3],
        [5, 2]]])

In [13]:
a = np.array([
    [[0,1,], 
     [3,4,],
     [3,4,]],

    [[2,3,], 
     [2,5,],
     [3,4,]],
    
    [[2,3,], 
     [4,5,],
     [3,4,]],
    
    [[2,3,], 
     [5,5,],
     [3,4,]],
])
a.shape

(4, 3, 2)

In [49]:
mu = np.expand_dims(np.mean(a, axis=(-1,-2)), axis=(-1,-2))
sigma = np.expand_dims(np.std(a, axis=(-1, -2)), axis=(-1, -2))

np.std((a - mu) / sigma, axis=(1,2))



array([1., 1., 1., 1.])

In [1]:
from library.datasets import Bootstrapped_Signal_Binned_Dataset

In [2]:
ds = Bootstrapped_Signal_Binned_Dataset("gen", "train", "../../state/new_physics/data/processed")

In [3]:
ds.generate(range(1,3), "../../state/new_physics/data/raw/signal", 100, 10)

In [4]:
ds.load()

In [5]:
ds.bin_values

array([-2.  , -1.93, -1.85, -1.78, -1.7 , -1.63, -1.56, -1.48, -1.41,
       -1.34, -1.26, -1.19, -1.11, -1.04, -0.97, -0.89, -0.82, -0.75,
       -0.67, -0.6 , -0.52, -0.45, -0.38, -0.3 , -0.23, -0.15, -0.08,
       -0.01,  0.  ,  0.07,  0.14,  0.21,  0.29,  0.36,  0.44,  0.51,
        0.58,  0.66,  0.73,  0.8 ,  0.88,  0.95,  1.03,  1.1 ])

In [16]:
ds.features.shape

torch.Size([440, 100, 4])

In [19]:
ds.labels

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,
         7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14,
        14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 22, 22, 22, 22, 